# Regime Prediction with Machine Learning

Based on the database below, which is updated regularly, we are going to build a recession predictor, testing various machine learning models in the process.  The database of monthly macro indicators was downloaded on February 5, 2020, and the most recent entries were for the month of December 2019.

Source Database:

- M. McCracken and S. Ng "FRED-MD: A Monthly Database for Macroeconomic Research", Working Paper, 2015. https://research.stlouisfed.org/econ/mccracken/fred-databases/
- https://s3.amazonaws.com/files.fred.stlouisfed.org/fred-md/monthly/current.csv

## Table of Contents:
&nbsp;&nbsp;1. [Set Up Environment and Read Data](#1)

&nbsp;&nbsp;2. [Data Cleaning](#2)


## 1. Set Up Environment and Read Data <a id="1"></a>

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

import requests
import csv
import os

import time
import datetime

# Anaconda has all these packages
from statsmodels.tsa.stattools import adfuller #to check unit root in time series  

# we don't use any of the following in this notebook
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel

import seaborn as sns #for correlation heatmap

import warnings
warnings.filterwarnings('ignore')

In [43]:
url = 'https://s3.amazonaws.com/files.fred.stlouisfed.org/fred-md/monthly/current.csv'
bigmacro=pd.read_csv(url)
bigmacro=bigmacro.rename(columns={'sasdate':'Date'})
bigmacro=bigmacro.iloc[1:,]
bigmacro=bigmacro.iloc[:-1,]

ts=time.localtime()
day= time.strftime('%Y-%m-%d', ts)
bigmacro.to_csv('current '+day+'.csv')
#bigmacro=bigmacro.reset_index()
bigmacro.tail()

,Date,RPI,W875RX1,DPCERA3M086SBEA,CMRMTSPLx,RETAILx,INDPRO,IPFPNSS,IPFINAL,IPCONGD,...,DSERRG3M086SBEA,CES0600000008,CES2000000008,CES3000000008,UMCSENTx,MZMSL,DTCOLNVHFNM,DTCTHFNM,INVEST,VXOCLSx
728,8/1/2019,17038.444,14131.9,121.340,1524456.0,526862.0,109.9634,104.7719,103.4415,105.4906,...,117.869,24.83,28.55,22.22,89.8,16453.4,318814.45,736714.08,3726.4488,21.0968
729,9/1/2019,17088.646,14172.8,121.554,1524004.0,524651.0,109.4437,104.0988,102.6215,104.7760,...,118.043,24.89,28.59,22.27,93.2,16580.6,318895.38,734464.35,3788.6873,16.3081
730,10/1/2019,17060.836,14140.7,121.556,1515625.0,526420.0,108.8532,103.7820,102.2877,104.6329,...,118.270,24.95,28.64,22.31,95.5,16757.7,319414.40,735219.31,3795.6857,15.9863
731,11/1/2019,17119.327,14198.3,121.968,1526868.0,527841.0,109.7573,105.0880,103.9617,106.3976,...,118.447,24.97,28.69,22.37,96.8,16903.0,319615.72,735871.89,3835.0809,12.6423
732,12/1/2019,17111.055,14197.1,122.030,NaN,529606.0,109.4330,104.5049,103.1292,105.1003,...,118.773,25.05,28.76,22.46,99.3,16979.0,NaN,NaN,3823.8975,13.3469


In [44]:
Recession_periods=pd.read_csv('Recession_Periods.csv')
regime = np.append(Recession_periods['Regime'].values,np.array(['Normal']*11))
bigmacro.insert(loc=1, column="Regime", value=regime)

In [45]:
bigmacro.tail()

,Date,Regime,RPI,W875RX1,DPCERA3M086SBEA,CMRMTSPLx,RETAILx,INDPRO,IPFPNSS,IPFINAL,...,DSERRG3M086SBEA,CES0600000008,CES2000000008,CES3000000008,UMCSENTx,MZMSL,DTCOLNVHFNM,DTCTHFNM,INVEST,VXOCLSx
728,8/1/2019,Normal,17038.444,14131.9,121.340,1524456.0,526862.0,109.9634,104.7719,103.4415,...,117.869,24.83,28.55,22.22,89.8,16453.4,318814.45,736714.08,3726.4488,21.0968
729,9/1/2019,Normal,17088.646,14172.8,121.554,1524004.0,524651.0,109.4437,104.0988,102.6215,...,118.043,24.89,28.59,22.27,93.2,16580.6,318895.38,734464.35,3788.6873,16.3081
730,10/1/2019,Normal,17060.836,14140.7,121.556,1515625.0,526420.0,108.8532,103.7820,102.2877,...,118.270,24.95,28.64,22.31,95.5,16757.7,319414.40,735219.31,3795.6857,15.9863
731,11/1/2019,Normal,17119.327,14198.3,121.968,1526868.0,527841.0,109.7573,105.0880,103.9617,...,118.447,24.97,28.69,22.37,96.8,16903.0,319615.72,735871.89,3835.0809,12.6423
732,12/1/2019,Normal,17111.055,14197.1,122.030,NaN,529606.0,109.4330,104.5049,103.1292,...,118.773,25.05,28.76,22.46,99.3,16979.0,NaN,NaN,3823.8975,13.3469


## 2. Data Cleaning <a id="2"></a>

We will follow the steps below to clean data and make it ready for feature selection process.

1. Remove the variables with missing observations
2. Add lags of the variables as additional features
3. Test stationarity of time series
4. Standardize the dataset

In [46]:
#remove columns with missing observations
missing_colnames=[]
print(bigmacro.shape)

for i in bigmacro.drop(['Date','Regime'],axis=1):
    observations=len(bigmacro)-bigmacro[i].count()
    if (observations>10):
        print(i+':'+str(observations))
        missing_colnames.append(i)

bigmacro=bigmacro.drop(labels=missing_colnames, axis=1)
#  there are a few rows with missing values but they are at the end of the dataset, so there are no missing months
#  in dataset; 59 years and 10 months, starting 1/1/1959, ending 10/2018, or 718 months
bigmacro=bigmacro.dropna(axis=0)

print(bigmacro.shape)
bigmacro.head()

(732, 130)
PERMIT:12
PERMITNE:12
PERMITMW:12
PERMITS:12
PERMITW:12
ACOGNO:398
ANDENOx:109
TWEXMMTH:168
UMCSENTx:154
VXOCLSx:42
(729, 120)


,Date,Regime,RPI,W875RX1,DPCERA3M086SBEA,CMRMTSPLx,RETAILx,INDPRO,IPFPNSS,IPFINAL,...,DDURRG3M086SBEA,DNDGRG3M086SBEA,DSERRG3M086SBEA,CES0600000008,CES2000000008,CES3000000008,MZMSL,DTCOLNVHFNM,DTCTHFNM,INVEST
1,1/1/1959,Normal,2437.296,2288.8,17.302,292258.8329,18235.77392,22.6250,23.4581,22.1904,...,56.918,17.791,11.358,2.13,2.45,2.04,274.9,6476.0,12298.0,84.2043
2,2/1/1959,Normal,2446.902,2297.0,17.482,294429.5453,18369.56308,23.0681,23.7747,22.3827,...,56.951,17.798,11.375,2.14,2.46,2.05,276.0,6476.0,12298.0,83.5280
3,3/1/1959,Normal,2462.689,2314.0,17.647,293425.3813,18523.05762,23.4004,23.9186,22.4925,...,57.022,17.785,11.395,2.15,2.45,2.07,277.4,6508.0,12349.0,81.6405
4,4/1/1959,Normal,2478.744,2330.3,17.584,299331.6505,18534.46600,23.8989,24.2641,22.8221,...,57.080,17.796,11.436,2.16,2.47,2.08,278.1,6620.0,12484.0,81.8099
5,5/1/1959,Normal,2493.228,2345.8,17.796,301372.9597,18679.66354,24.2589,24.4655,23.0418,...,57.175,17.777,11.454,2.17,2.48,2.08,280.1,6753.0,12646.0,80.7315


In [47]:
# bigmacro Date ended at 9/1/2019 after cleaning out rows with na's.

# Add lags
for col in bigmacro.drop(['Date', 'Regime'], axis=1):
    for n in [3,6,9,12,18]:
        bigmacro['{} {}M lag'.format(col, n)] = bigmacro[col].shift(n).ffill().values 

# 1 month ahead prediction
bigmacro["Regime"] = bigmacro["Regime"].shift(-1)

bigmacro=bigmacro.dropna(axis=0)
bigmacro.tail(1)
# now only goes to 8/2019, due to shifting regime back one month
# 710 columns vs. 118 data columns before adding lags, 5x118 => 590+120=710

,Date,Regime,RPI,W875RX1,DPCERA3M086SBEA,CMRMTSPLx,RETAILx,INDPRO,IPFPNSS,IPFINAL,...,DTCTHFNM 3M lag,DTCTHFNM 6M lag,DTCTHFNM 9M lag,DTCTHFNM 12M lag,DTCTHFNM 18M lag,INVEST 3M lag,INVEST 6M lag,INVEST 9M lag,INVEST 12M lag,INVEST 18M lag
728,8/1/2019,Normal,17038.444,14131.9,121.34,1524456.0,526862.0,109.9634,104.7719,103.4415,...,732396.28,729307.47,734086.61,735979.69,736073.92,3626.6663,3540.6859,3448.43,3422.8354,3401.0419


In [48]:
#120 columns before, or 118 minus the Date and Regime
# 118x5 lags = 590 additional columns; 590+120 = 710
print(bigmacro.shape)
# only 699 rows is 19 months less than before, because shifts clipped off 18 months from beginning of time series due to 
#largest time lag, and one month off end of time series
bigmacro['Date'].iloc[[0,-1]]


(710, 710)


19     7/1/1960
728    8/1/2019
Name: Date, dtype: object

Augmented Dickey-Fuller Test can be used to test for stationarity in macroeconomic time series variables. We will use `adfuller` function from `statsmodels` module in Python. More information about the function can be found __[here](https://www.statsmodels.org/dev/generated/statsmodels.tsa.stattools.adfuller.html)__.

In [49]:
#check stationarity
from statsmodels.tsa.stattools import adfuller #to check unit root in time series 
threshold=0.01 #significance level
for column in bigmacro.drop(['Date','Regime'], axis=1):
    result=adfuller(bigmacro[column])
    if result[1]>threshold:
        print(column)
        bigmacro[column]=bigmacro[column].diff()  # replaces values with diff between current and prior row value
bigmacro=bigmacro.dropna(axis=0)

RPI
W875RX1
DPCERA3M086SBEA
CMRMTSPLx
RETAILx
INDPRO
IPFPNSS
IPFINAL
IPCONGD
IPDCONGD
IPNCONGD
IPBUSEQ
IPMAT
IPDMAT
IPNMAT
IPMANSICS
IPB51222S
IPFUELS
HWI
CLF16OV
CE16OV
UNRATE
UEMPMEAN
UEMPLT5
UEMP5TO14
UEMP15OV
UEMP15T26
UEMP27OV
CLAIMSx
PAYEMS
USGOOD
CES1021000001
USCONS
MANEMP
DMANEMP
NDMANEMP
SRVPRD
USTPU
USWTRADE
USTRADE
USFIRE
USGOVT
CES0600000007
AWOTMAN
AWHMAN
HOUSTNE
HOUSTMW
AMDMNOx
AMDMUOx
BUSINVx
ISRATIOx
M1SL
M2SL
M2REAL
BOGMBASE
TOTRESNS
NONBORRES
BUSLOANS
REALLN
NONREVSL
CONSPI
S&P 500
S&P: indust
S&P div yield
FEDFUNDS
CP3Mx
TB3MS
TB6MS
GS1
GS5
GS10
AAA
BAA
EXSZUSx
EXJPUSx
EXUSUKx
EXCAUSx
WPSFD49207
WPSFD49502
WPSID61
WPSID62
OILPRICEx
PPICMM
CPIAUCSL
CPIAPPSL
CPITRNSL
CPIMEDSL
CUSR0000SAC
CUSR0000SAD
CUSR0000SAS
CPIULFSL
CUSR0000SA0L2
CUSR0000SA0L5
PCEPI
DDURRG3M086SBEA
DNDGRG3M086SBEA
DSERRG3M086SBEA
CES0600000008
CES2000000008
CES3000000008
MZMSL
DTCOLNVHFNM
DTCTHFNM
INVEST
RPI 3M lag
RPI 6M lag
RPI 9M lag
RPI 12M lag
RPI 18M lag
W875RX1 3M lag
W875RX1 6M lag
W875RX1

DDURRG3M086SBEA 12M lag
DDURRG3M086SBEA 18M lag
DNDGRG3M086SBEA 3M lag
DNDGRG3M086SBEA 6M lag
DNDGRG3M086SBEA 9M lag
DNDGRG3M086SBEA 12M lag
DNDGRG3M086SBEA 18M lag
DSERRG3M086SBEA 3M lag
DSERRG3M086SBEA 6M lag
DSERRG3M086SBEA 9M lag
DSERRG3M086SBEA 12M lag
DSERRG3M086SBEA 18M lag
CES0600000008 3M lag
CES0600000008 6M lag
CES0600000008 9M lag
CES0600000008 12M lag
CES0600000008 18M lag
CES2000000008 3M lag
CES2000000008 6M lag
CES2000000008 9M lag
CES2000000008 12M lag
CES2000000008 18M lag
CES3000000008 3M lag
CES3000000008 6M lag
CES3000000008 9M lag
CES3000000008 12M lag
CES3000000008 18M lag
MZMSL 3M lag
MZMSL 6M lag
MZMSL 9M lag
MZMSL 12M lag
MZMSL 18M lag
DTCOLNVHFNM 3M lag
DTCOLNVHFNM 6M lag
DTCOLNVHFNM 9M lag
DTCOLNVHFNM 12M lag
DTCOLNVHFNM 18M lag
DTCTHFNM 3M lag
DTCTHFNM 6M lag
DTCTHFNM 9M lag
DTCTHFNM 12M lag
DTCTHFNM 18M lag
INVEST 3M lag
INVEST 6M lag
INVEST 9M lag
INVEST 12M lag
INVEST 18M lag


In [50]:
threshold=0.01 #significance level
for column in bigmacro.drop(['Date','Regime'], axis=1):
    result=adfuller(bigmacro[column])
    if result[1]>threshold:
        print(column)
        bigmacro[column]=bigmacro[column].diff()
bigmacro=bigmacro.dropna(axis=0)

M1SL
M2SL
REALLN
NONREVSL
CPIAPPSL
CPIMEDSL
CUSR0000SAD
CUSR0000SAS
DDURRG3M086SBEA
DSERRG3M086SBEA
CES0600000008
CES2000000008
CES3000000008
MZMSL
INVEST
IPNCONGD 6M lag
CUMFNS 3M lag
M1SL 3M lag
M1SL 6M lag
M1SL 9M lag
M1SL 12M lag
M1SL 18M lag
M2SL 3M lag
M2SL 6M lag
M2SL 9M lag
M2SL 12M lag
M2SL 18M lag
REALLN 3M lag
REALLN 6M lag
REALLN 9M lag
REALLN 12M lag
REALLN 18M lag
NONREVSL 3M lag
NONREVSL 6M lag
NONREVSL 9M lag
NONREVSL 12M lag
NONREVSL 18M lag
CPIAPPSL 3M lag
CPIAPPSL 6M lag
CPIAPPSL 9M lag
CPIAPPSL 12M lag
CPIAPPSL 18M lag
CPIMEDSL 3M lag
CPIMEDSL 6M lag
CPIMEDSL 9M lag
CPIMEDSL 12M lag
CPIMEDSL 18M lag
CUSR0000SAD 3M lag
CUSR0000SAD 6M lag
CUSR0000SAD 9M lag
CUSR0000SAD 12M lag
CUSR0000SAD 18M lag
CUSR0000SAS 3M lag
CUSR0000SAS 6M lag
CUSR0000SAS 9M lag
CUSR0000SAS 12M lag
CUSR0000SAS 18M lag
PCEPI 9M lag
PCEPI 12M lag
PCEPI 18M lag
DDURRG3M086SBEA 3M lag
DDURRG3M086SBEA 6M lag
DDURRG3M086SBEA 9M lag
DDURRG3M086SBEA 12M lag
DDURRG3M086SBEA 18M lag
DSERRG3M086SBEA 3M la

In [51]:
threshold=0.01 #significance level
for column in bigmacro.drop(['Date','Regime'], axis=1):
    result=adfuller(bigmacro[column])
    if result[1]>threshold:
        print(column)
bigmacro=bigmacro.dropna(axis=0)      
# not sure why we do this three times, but we do get just zero columns that are still 
# nonstationary after this 

In [52]:
print(bigmacro.shape)   #Still 710 columns but we have lost two rows because taking .diff() results in one NA after each operation
# from start of series
bigmacro['Date'].iloc[[0,-1]]


(708, 710)


21     9/1/1960
728    8/1/2019
Name: Date, dtype: object

In [53]:
# Standardize
from sklearn.preprocessing import StandardScaler
features=bigmacro.drop(['Date','Regime'],axis=1)
col_names=features.columns

scaler=StandardScaler()
scaler.fit(features)
standardized_features=scaler.transform(features)
# features have been centered and scaled

print(standardized_features.shape)
df = pd.DataFrame(data=standardized_features, columns=col_names)
df.insert(loc=0, column="Date", value=bigmacro['Date'].values)
df.insert(loc=1, column='Regime', value=bigmacro['Regime'].values)
df.shape

(708, 708)


(708, 710)

In [54]:
bigmacro.head()

,Date,Regime,RPI,W875RX1,DPCERA3M086SBEA,CMRMTSPLx,RETAILx,INDPRO,IPFPNSS,IPFINAL,...,DTCTHFNM 3M lag,DTCTHFNM 6M lag,DTCTHFNM 9M lag,DTCTHFNM 12M lag,DTCTHFNM 18M lag,INVEST 3M lag,INVEST 6M lag,INVEST 9M lag,INVEST 12M lag,INVEST 18M lag
21,9/1/1960,Recession,3.275,0.7,0.087,4748.0925,-17.63113,-0.2492,-0.2015,-0.1373,...,257.0,95.0,169.0,186.0,51.0,-0.7134,0.3085,0.2918,-0.4119,-1.8875
22,10/1/1960,Recession,7.607,7.6,0.089,-5606.3790,165.94004,-0.0277,0.1439,0.1098,...,212.0,174.0,5.0,147.0,135.0,2.3150,1.7489,-0.7196,-0.6691,0.1694
23,11/1/1960,Recession,-11.052,-13.9,-0.064,-2854.8054,-271.72682,-0.3323,-0.2590,-0.2746,...,180.0,146.0,81.0,85.0,162.0,-1.7373,0.2718,-2.1152,-0.3442,-1.0784
24,12/1/1960,Recession,-11.710,-14.1,-0.225,2270.7833,-136.90054,-0.4431,-0.3454,-0.3021,...,74.0,257.0,95.0,169.0,280.0,0.8476,-0.7134,-1.8067,0.2918,-2.0343
25,1/1/1961,Normal,22.912,20.9,0.044,-10337.0224,14.51975,0.0277,-0.0288,-0.0550,...,-6.0,212.0,174.0,5.0,273.0,0.9454,2.3150,-0.0578,-0.7196,0.3012


In [81]:
# Note how the big macro at this point starts with index 21, at 9/1/60, whereas at the beginning of this notebook, it 
# started at 1/1/59

In [56]:
df.to_csv('current_cleaned_'+day+'.csv', index=False)
